## Comparison of the total output per technology considering TimeStep equal to 3 and 4

In [1]:
import altair as alt
import pandas as pd
from openTEPES.openTEPES import openTEPES_run

### Defining the DirName, CaseName, SolverName, and IndLogConsole.

In [2]:
DirName =''
CaseName = '9n_ByStages'
SolverName = 'appsi_highs'
IndLogConsole = 'No'

In the folder called ``9n_ByStages``, you can find the CSV file ``oT_Data_Parameter_9n_ByStages`` which is also shown as follows:

In [3]:
df = pd.read_csv(CaseName+'/oT_Data_Parameter_'+CaseName+'.csv', index_col=[0])
df.head()

,ENSCost,CO2Cost,UpReserveActivation,DwReserveActivation,MinRatioDwUp,MaxRatioDwUp,SBase,ReferenceNode,TimeStep
Parameters,10000,25,0.25,0.3,0,1,100,Node_4,3


This CSV file comprises all the system parameters of the openTEPES mode. It has a column called TimeStep, that having ``1`` for hourly resolution, ``2`` for bi-hourly resolution, etc. In our example, we consider a TimeStep equal to ``3`` (tri-hourly resolution) and equal to ``4`` (tetra-hourly resolution).

### Having tri-hourly resolution

In [4]:
df['TimeStep'] = 3

In [5]:
df.to_csv(CaseName+'/oT_Data_Parameter_'+CaseName+'.csv', index=True)

In [6]:
model_TS3 = openTEPES_run(DirName, CaseName, SolverName, IndLogConsole)

Input data                             ****
Reading    input data                  ...  0 s


C:\Users\erik_\miniconda3\lib\site-packages\pyomo\core\base\block.py:541: PerformanceWarning: indexing past lexsort depth may impact performance.
  self.add_component(name, val)


Setting up input data                  ...  7 s
Setting up variables                   ...  3 s
Investment           model formulation ****
Generation operation model formulation ****
Network    switching model formulation ****
Network    operation model formulation ****
Writing LP file                        ...  24 s
Problem solving                        ****
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x570769
  Lower bound: 141.10272176111857
  Upper bound: 141.10272176111857
  Number of objectives: 1
  Number of constraints: 285373
  Number of variables: 363532
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 363532
  Number of nonzer

### Having tetra-hourly resolution

In [7]:
df['TimeStep'] = 4

In [8]:
df.to_csv(CaseName+'/oT_Data_Parameter_'+CaseName+'.csv', index=True)

In [9]:
model_TS4 = openTEPES_run(DirName, CaseName, SolverName, IndLogConsole)

Input data                             ****
Reading    input data                  ...  0 s


C:\Users\erik_\miniconda3\lib\site-packages\pyomo\core\base\block.py:541: PerformanceWarning:

indexing past lexsort depth may impact performance.



Setting up input data                  ...  5 s
Setting up variables                   ...  2 s
Investment           model formulation ****
Generation operation model formulation ****
Network    switching model formulation ****
Network    operation model formulation ****
Writing LP file                        ...  15 s
Problem solving                        ****
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x428081
  Lower bound: 142.15017491480492
  Upper bound: 142.15017491480492
  Number of objectives: 1
  Number of constraints: 214031
  Number of variables: 272626
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 272626
  Number of nonzer

### Getting the total output per technology

In [10]:
OutputToFile_TS3 = pd.Series(data=[sum(model_TS3.vTotalOutput[sc,p,n,g]() for g in model_TS3.g if (gt,g) in model_TS3.t2g)*model_TS3.pTimeStep() for sc,p,n,gt in model_TS3.sc*model_TS3.p*model_TS3.n*model_TS3.gt], index=pd.MultiIndex.from_tuples(model_TS3.sc*model_TS3.p*model_TS3.n*model_TS3.gt))
OutputToFile_TS3 = OutputToFile_TS3.to_frame(name='GW' ).reset_index().pivot_table(index=['level_0','level_1','level_2'], columns='level_3', values='GW').rename_axis(['Scenario','Period','LoadLevel'], axis=0).rename_axis([None], axis=1).sum(axis=0)

In [11]:
OutputToFile_TS4 = pd.Series(data=[sum(model_TS4.vTotalOutput[sc,p,n,g]() for g in model_TS4.g if (gt,g) in model_TS4.t2g)*model_TS4.pTimeStep() for sc,p,n,gt in model_TS4.sc*model_TS4.p*model_TS4.n*model_TS4.gt], index=pd.MultiIndex.from_tuples(model_TS4.sc*model_TS4.p*model_TS4.n*model_TS4.gt))
OutputToFile_TS4 = OutputToFile_TS4.to_frame(name='GW' ).reset_index().pivot_table(index=['level_0','level_1','level_2'], columns='level_3', values='GW').rename_axis(['Scenario','Period','LoadLevel'], axis=0).rename_axis([None], axis=1).sum(axis=0)

In [12]:
OutputToFile_TS3.index.names = ['Technology']
OutputToFile_TS3 = OutputToFile_TS3.to_frame(name='GWh')
OutputToFile_TS3['Resolution'] = 'TimeStep-3'
OutputToFile_TS3 = OutputToFile_TS3.reset_index()
OutputToFile_TS4.index.names = ['Technology']
OutputToFile_TS4 = OutputToFile_TS4.to_frame(name='GWh')
OutputToFile_TS4['Resolution'] = 'TimeStep-4'
OutputToFile_TS4 = OutputToFile_TS4.reset_index()

In [13]:
OutputResults = pd.concat([OutputToFile_TS3, OutputToFile_TS4], axis=0)
OutputResults['GWh'] = round(OutputResults['GWh'], 2)

In [14]:
chart = alt.Chart(OutputResults).mark_bar().encode(x='Resolution:O', y='GWh:Q', color='Resolution:O', column='Technology:N')
# chart.save(CaseName+'/oT_Plot_TechnologyOutputComparison_'+CaseName+'.html', embed_options={'renderer':'svg'})

In [15]:
chart

alt.Chart(...)